# Model Comparison
## Random Under Sampling

### XGBoost - Original Data set
### Random Under Sampling- Logistic regression, Random Forest, XGBoost


In [1]:
#import libraries
import pandas as pd
import numpy as np

In [2]:
#loading data
df = pd.read_csv(r"C:\Users\Pulani\Desktop\NTUCLHUB\MYPROJECT_CAPSTONE\Project 1 - Finance - Predictive Analytics\Dataset\loan.csv")
print ("Data loaded successfully!")

# Display the first few rows of the dataset
df.head()

Data loaded successfully!


,customer_id,loan_id,loan_type,loan_amount,interest_rate,loan_term,employment_type,income_level,credit_score,gender,marital_status,education_level,application_date,approval_date,disbursement_date,due_date,default_status
0,CUST-00004912,LN00004170,Car Loan,16795,0.051852,15,Self-employed,Medium,833,Male,Single,Master,05-04-2018,23-04-2018,24-04-2018,14-08-2018,False
1,CUST-00004194,LN00002413,Personal Loan,1860,0.089296,56,Full-time,Medium,776,Female,Married,Bachelor,30-12-2022,31-12-2022,12-01-2023,05-04-2023,False
2,CUST-00003610,LN00000024,Personal Loan,77820,0.070470,51,Full-time,Low,697,Male,Divorced,High School,15-11-2019,18-11-2019,27-11-2019,24-02-2020,False
3,CUST-00001895,LN00001742,Car Loan,55886,0.062155,30,Full-time,Low,795,Female,Married,PhD,25-08-2021,08-09-2021,11-09-2021,25-02-2022,False
4,CUST-00003782,LN00003161,Home Loan,7265,0.070635,48,Part-time,Low,519,Female,Married,High School,02-09-2020,07-09-2020,11-09-2020,29-12-2020,False


In [5]:
# Set the reference date to January 1, 1970 (Unix Epoch)
reference_date = pd.to_datetime('1970-01-01')

# Convert date columns to datetime format, with day first
df['application_date'] = pd.to_datetime(df['application_date'], dayfirst=True)
df['approval_date'] = pd.to_datetime(df['approval_date'], dayfirst=True)
df['disbursement_date'] = pd.to_datetime(df['disbursement_date'], dayfirst=True)
df['due_date'] = pd.to_datetime(df['due_date'], dayfirst=True)

# Calculate the number of days since the reference date (Unix Epoch)
df['application_days'] = (df['application_date'] - reference_date).dt.days
df['approval_days'] = (df['approval_date'] - reference_date).dt.days
df['disbursement_days'] = (df['disbursement_date'] - reference_date).dt.days
df['due_days'] = (df['due_date'] - reference_date).dt.days

# Drop the original date columns
df = df.drop(['application_date', 'approval_date', 'disbursement_date', 'due_date'], axis=1)

# Check the updated DataFrame
print("Updated DataFrame after processing date columns:")
df.head()

Updated DataFrame after processing date columns:


,customer_id,loan_id,loan_type,loan_amount,interest_rate,loan_term,employment_type,income_level,credit_score,gender,marital_status,education_level,default_status,application_days,approval_days,disbursement_days,due_days
0,CUST-00004912,LN00004170,Car Loan,16795,0.051852,15,Self-employed,Medium,833,Male,Single,Master,False,17626,17644,17645,17757
1,CUST-00004194,LN00002413,Personal Loan,1860,0.089296,56,Full-time,Medium,776,Female,Married,Bachelor,False,19356,19357,19369,19452
2,CUST-00003610,LN00000024,Personal Loan,77820,0.070470,51,Full-time,Low,697,Male,Divorced,High School,False,18215,18218,18227,18316
3,CUST-00001895,LN00001742,Car Loan,55886,0.062155,30,Full-time,Low,795,Female,Married,PhD,False,18864,18878,18881,19048
4,CUST-00003782,LN00003161,Home Loan,7265,0.070635,48,Part-time,Low,519,Female,Married,High School,False,18507,18512,18516,18625


In [7]:
# Drop the customer_id and loan_id columns
df = df.drop(['customer_id', 'loan_id'], axis=1)

# Check the updated DataFrame
print("Updated DataFrame after dropping customer_id and loan_id:")
df.head()

Updated DataFrame after dropping customer_id and loan_id:


,loan_type,loan_amount,interest_rate,loan_term,employment_type,income_level,credit_score,gender,marital_status,education_level,default_status,application_days,approval_days,disbursement_days,due_days
0,Car Loan,16795,0.051852,15,Self-employed,Medium,833,Male,Single,Master,False,17626,17644,17645,17757
1,Personal Loan,1860,0.089296,56,Full-time,Medium,776,Female,Married,Bachelor,False,19356,19357,19369,19452
2,Personal Loan,77820,0.070470,51,Full-time,Low,697,Male,Divorced,High School,False,18215,18218,18227,18316
3,Car Loan,55886,0.062155,30,Full-time,Low,795,Female,Married,PhD,False,18864,18878,18881,19048
4,Home Loan,7265,0.070635,48,Part-time,Low,519,Female,Married,High School,False,18507,18512,18516,18625


In [9]:
from sklearn.preprocessing import LabelEncoder

# Initialize separate LabelEncoders for each column
income_encoder = LabelEncoder()
education_encoder = LabelEncoder()

# Apply LabelEncoder to income_level and education_level columns
df['income_level'] = income_encoder.fit_transform(df['income_level']) + 1
df['education_level'] = education_encoder.fit_transform(df['education_level']) + 1

# Optional: Print the mappings for verification
income_mapping = {label: num + 1 for num, label in enumerate(income_encoder.classes_)}
education_mapping = {label: num + 1 for num, label in enumerate(education_encoder.classes_)}

print("Income Level Mapping:")
print(income_mapping)

print("Education Level Mapping:")
print(education_mapping)

Income Level Mapping:
{'High': 1, 'Low': 2, 'Medium': 3}
Education Level Mapping:
{'Bachelor': 1, 'High School': 2, 'Master': 3, 'PhD': 4}


In [11]:
# Convert default_status (boolean) to int
df['default_status'] = df['default_status'].astype(int)

# One-hot encode remaining categorical columns
# Assuming the other categorical columns are 'loan_type', 'employment_type', 'gender', 'marital_status'
df = pd.get_dummies(df, columns=['loan_type', 'employment_type', 'gender', 'marital_status'], drop_first=True, dtype=int)

In [13]:
# Check the updated DataFrame
print("Updated DataFrame after one-hot encoding:")
df.head()

Updated DataFrame after one-hot encoding:


,loan_amount,interest_rate,loan_term,income_level,credit_score,education_level,default_status,application_days,approval_days,disbursement_days,due_days,loan_type_Education Loan,loan_type_Home Loan,loan_type_Personal Loan,employment_type_Part-time,employment_type_Self-employed,gender_Male,marital_status_Married,marital_status_Single
0,16795,0.051852,15,3,833,3,0,17626,17644,17645,17757,0,0,0,0,1,1,0,1
1,1860,0.089296,56,3,776,1,0,19356,19357,19369,19452,0,0,1,0,0,0,1,0
2,77820,0.070470,51,2,697,2,0,18215,18218,18227,18316,0,0,1,0,0,1,0,0
3,55886,0.062155,30,2,795,4,0,18864,18878,18881,19048,0,0,0,0,0,0,1,0
4,7265,0.070635,48,2,519,2,0,18507,18512,18516,18625,0,1,0,1,0,0,1,0


In [15]:
# Import required libraries for machine learning models and evaluation
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [17]:
from sklearn.model_selection import train_test_split

# Split the data into features (X) and target (y)
X = df.drop('default_status', axis=1)  # Features (drop the target column)
y = df['default_status']  # Target (the column we want to predict)

# Perform the train-test split (80% for training, 20% for testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the shape of the training and testing sets
print("Training set shape (X_train, y_train):", X_train.shape, y_train.shape)
print("Testing set shape (X_test, y_test):", X_test.shape, y_test.shape)

Training set shape (X_train, y_train): (4000, 18) (4000,)
Testing set shape (X_test, y_test): (1000, 18) (1000,)


In [36]:
# Importing necessary libraries
# XGBoost 80/20:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Assuming 'data' is your DataFrame
# Separating features (X) and the target (y)
X = df.drop('default_status', axis=1)  # Features
y = df['default_status']  # Target

# Splitting the dataset into train and test sets (80/20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# Creating the XGBoost model
xgb_model = XGBClassifier(random_state=42, scale_pos_weight=len(y_train[y_train == 0]) / len(y_train[y_train == 1]))

# Fitting the model
xgb_model.fit(X_train, y_train)

# Predicting on the test set
y_pred = xgb_model.predict(X_test)

# Evaluating the model with classification report and confusion matrix

# Print the confusion matrix and classification report
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Confusion Matrix:
[[1017  183]
 [ 257   43]]

Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.85      0.82      1200
           1       0.19      0.14      0.16       300

    accuracy                           0.71      1500
   macro avg       0.49      0.50      0.49      1500
weighted avg       0.68      0.71      0.69      1500



In [21]:
# Check class distribution 
print("Class distribution: ")
print(y.value_counts())

Class distribution: 
default_status
0    4001
1     999
Name: count, dtype: int64


In [3]:
#Ranodom UNERSAMPLING Logistic regression


# Import necessary libraries
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from collections import Counter

# Define the features (X) and target (y)
X = df.drop('default_status', axis=1)
y = df['default_status']

# Split the dataset into training and testing sets (80/20 split as per your preference)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Apply Random UnderSampler to the training set to undersample the majority class
undersampler = RandomUnderSampler(random_state=42)
X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train, y_train)

# Check class distribution after undersampling
print("\nClass distribution after undersampling:")
print(Counter(y_train_resampled))

# Standardize the feature columns
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_resampled)
X_test_scaled = scaler.transform(X_test)

# Create and train the Logistic Regression model
log_reg = LogisticRegression(max_iter=5000, class_weight='balanced', random_state=42)
log_reg.fit(X_train_scaled, y_train_resampled)

# Make predictions on the test data
y_pred = log_reg.predict(X_test_scaled)

# Print the confusion matrix and classification report
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

NameError: name 'df' is not defined

In [27]:
#Ranodom UNERSAMPLING Random Forest
# Import necessary libraries
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from collections import Counter

# Define the features (X) and target (y)
X = df.drop('default_status', axis=1)
y = df['default_status']

# Split the dataset into training and testing sets (80/20 split as per your preference)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Apply Random UnderSampler to the training set to undersample the majority class
undersampler = RandomUnderSampler(random_state=42)
X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train, y_train)

# Check class distribution after undersampling
print("Class distribution after undersampling:")
print(Counter(y_train_resampled))

# Create and train the Random Forest model
rf_model = RandomForestClassifier(random_state=42, class_weight='balanced')
rf_model.fit(X_train_resampled, y_train_resampled)

# Make predictions on the test data
y_pred = rf_model.predict(X_test)

# Print the confusion matrix and classification report
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Class distribution after undersampling:
Counter({0: 799, 1: 799})
Confusion Matrix:
[[426 374]
 [113  87]]

Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.53      0.64       800
           1       0.19      0.43      0.26       200

    accuracy                           0.51      1000
   macro avg       0.49      0.48      0.45      1000
weighted avg       0.67      0.51      0.56      1000



In [31]:
#Ranodom UNERSAMPLING XGBoost

# Import necessary libraries
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix
from collections import Counter

# Define the features (X) and target (y)
X = df.drop('default_status', axis=1)
y = df['default_status']

# Split the dataset into training and testing sets (80/20 split as per your preference)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Apply Random UnderSampler to the training set to undersample the majority class
undersampler = RandomUnderSampler(random_state=42)
X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train, y_train)

# Check class distribution after undersampling
print("Class distribution after undersampling:")
print(Counter(y_train_resampled))

# Create and train the XGBoost model
xgb_model = XGBClassifier(random_state=42, scale_pos_weight=len(y_train_resampled[y_train_resampled == 0]) / len(y_train_resampled[y_train_resampled == 1]))
xgb_model.fit(X_train_resampled, y_train_resampled)

# Make predictions on the test data
y_pred = xgb_model.predict(X_test)

# Print the confusion matrix and classification report
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Class distribution after undersampling:
Counter({0: 799, 1: 799})
Confusion Matrix:
[[388 412]
 [111  89]]

Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.48      0.60       800
           1       0.18      0.45      0.25       200

    accuracy                           0.48      1000
   macro avg       0.48      0.46      0.43      1000
weighted avg       0.66      0.48      0.53      1000



### Model Comparison after Random Under Sampling

In [33]:
# Import necessary libraries
import pandas as pd

# Create the table for model comparison based on the provided results
data_undersampling_full = {
    'Model': ['Logistic Regression', 'Random Forest', 'XGBoost'],
    'Accuracy': [0.52, 0.51, 0.48],
    'Precision (Class 0)': [0.80, 0.79, 0.78],
    'Recall (Class 0)': [0.54, 0.53, 0.48],
    'F1-Score (Class 0)': [0.64, 0.64, 0.60],
    'Precision (Class 1)': [0.20, 0.19, 0.18],
    'Recall (Class 1)': [0.47, 0.43, 0.45],
    'F1-Score (Class 1)': [0.28, 0.26, 0.25]
}

# Create a DataFrame
df_undersampling_full_results = pd.DataFrame(data_undersampling_full)

# Display the DataFrame
df_undersampling_full_results


,Model,Accuracy,Precision (Class 0),Recall (Class 0),F1-Score (Class 0),Precision (Class 1),Recall (Class 1),F1-Score (Class 1)
0,Logistic Regression,0.52,0.80,0.54,0.64,0.20,0.47,0.28
1,Random Forest,0.51,0.79,0.53,0.64,0.19,0.43,0.26
2,XGBoost,0.48,0.78,0.48,0.60,0.18,0.45,0.25


**By comparing results:**

- Logistic Regression performs the best overall in terms of balancing accuracy and performance for both classes. While it doesn’t perform well for defaulters (Class 1), it has the highest accuracy and best handles non-defaulters (Class 0).

- Random Forest has slightly better recall for defaulters (Class 1) compared to Logistic Regression, but its performance is quite close overall.